# Übernahme der Daten aus OSM Protobuf, da hier alle Tages enthalten sind

In [ ]:
import duckdb
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import requests

## Url zurGeofabrik und lokale Ablage (zur Beschleunigung)

In [ ]:
url = 'https://download.geofabrik.de/europe/germany/niedersachsen-latest.osm.pbf'
output_path = "downloads/niedersachsen-latest.osm.pbf"

In [ ]:
# Download a PDF file to the downloads folder
response = requests.get(url)
with open(output_path, "wb") as f:
    f.write(response.content)
print(f"Downloaded file to {output_path}")

In [ ]:
duck = duckdb.connect(database=':memory:')

In [ ]:
duck.sql("""INSTALL spatial;
            LOAD spatial;"""
         )

In [ ]:
duck.sql(f"DESCRIBE SELECT * FROM ST_READOSM('{output_path}');")

In [ ]:
duck.sql(f"""create or replace table matching_ways as 
         select 
         id, tags['name'] as name, 
         tags['highway'] as highway, tags['maxspeed'] as maxspeed, tags['oneway'] as oneway, tags['oneway:bus'] as oneway_bus, tags['area'] as area,  tags['access'] as access, refs
        
         from ST_READOSM('{output_path}')
          where tags['highway'] is not null and kind = 'way'
          -- LIMIT 500;         
         """)

In [ ]:
duck.sql("""from matching_ways""")

In [ ]:
duck.sql(f"""
         CREATE OR REPLACE TEMP TABLE matching_ways_with_nodes_refs AS
SELECT id, UNNEST(refs) as ref, UNNEST(range(length(refs))) as ref_idx
FROM ST_READOSM('{output_path}')
SEMI JOIN matching_ways USING (id)
WHERE kind = 'way';

SELECT * FROM matching_ways_with_nodes_refs order by id, ref_idx;
         """)

In [ ]:
duck.sql(f"""CREATE or replace TEMP TABLE required_nodes_with_geometries AS
SELECT id, ST_POINT(lon, lat) geometry
FROM ST_READOSM('{output_path}') nodes
SEMI JOIN matching_ways_with_nodes_refs
ON nodes.id = matching_ways_with_nodes_refs.ref
WHERE kind = 'node';

SELECT * FROM required_nodes_with_geometries;""")

In [ ]:
duck.sql("""CREATE or replace TEMP TABLE matching_ways_linestrings AS
SELECT
    matching_ways.id,
    matching_ways.name,
         matching_ways.highway,
            matching_ways.maxspeed,
            matching_ways.oneway,
            matching_ways.oneway_bus,
            matching_ways.area,
            matching_ways.access,
    st_transform(ST_MakeLine(list(nodes.geometry ORDER BY ref_idx ASC)), 'EPSG:4326', 'EPSG:25832', always_xy:=true) linestring
FROM matching_ways
JOIN matching_ways_with_nodes_refs
ON matching_ways.id = matching_ways_with_nodes_refs.id
JOIN required_nodes_with_geometries nodes
ON matching_ways_with_nodes_refs.ref = nodes.id
GROUP BY 1, 2, 3,4,5,6,7,8;

SELECT * FROM matching_ways_linestrings;""")

### Auflisten der GDAL Treiber

In [ ]:
duck.sql("from st_drivers() order by short_name").df()

### Export als FileGDB

In [ ]:
duck.sql("""copy matching_ways_linestrings to 'out/niedersachsen_highways.gdb' (FORMAT gdal, 
            DRIVER 'OpenFileGDB', 
         SRS 'EPSG:25832', GEOMETRY_TYPE 'LINESTRING');""")

In [ ]:
duck.close()